<a href="https://colab.research.google.com/github/HeTalksInMaths/it-cert-automation-practice/blob/master/QOSF_Task_2_Final_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We make use of the pennylane framework for our task to learn parametrs in our quantum circuit to produce 01 and 10 with 1/2 probability each.

In [1]:
# !pip install pennylane   # uncomment first hash for installation on Colab
import pennylane as qml
from pennylane import numpy as np

We set the analytic parameter in our device initialization to False to introduce stochastisity in our measurements and later make use of dev.shots to adjust the number of measurements.


In [2]:
dev = qml.device("default.qubit", wires=2, analytic = False)

Let us start with a simple structure where we have expected results so that the focus is on developing intutition in the parameter learning. With two RY gates (with both qubits inititalized as 0) and a subsequent CNOT between the wires one expected result would be for the parameters to equal [pi/2, pi]. 


RY(pi/2) sends the first qubit from 0 to + (as is usually prescribed for creating a Bell State before application of the CNOT gate) and RY(pi) send the second qubit from 0 to 1. The subsequent CNOT then results in mismatched qubit values.

In [75]:
@qml.qnode(dev)
def circuit(params):
  qml.RY(params[0], wires=0)
  qml.RY(params[1], wires=1)
  qml.CNOT(wires=[0,1])
  return qml.probs(wires=[0,1])

We set the seed for reprodcubility and test our prior intution regarding the parameters with a large sample size (to get close to the theoeretical limit).

We also draw the circuit for clarity.

In [85]:
np.random.seed(0)
dev.shots = 10**6
circuit([np.pi/2, np.pi])

array([0.      , 0.499195, 0.500805, 0.      ])

In [89]:
print(circuit.draw())

 0: ──RY(1.571)──╭C──╭┤ Probs 
 1: ──RY(3.142)──╰X──╰┤ Probs 



Our intuition is correct and with the parameters [pi, pi/2] we are very close (but not identical due to sampling and stochasticity) to 0.5 probability for the 01 and 10 states. 

Note the probabilities are given to 6 decimals due to dev.shots = 10**6. We use qml.probs instead of qml.sample to save on some data processing but show below how qm.probs is noisy and is given to accuracy given by the number of samples set by dev.shots.

In [72]:
for i in [1, 10, 100, 1000]:
  dev.shots = i
  for j in [0,2]:
    np.random.seed(j)
    print("For {} samples with seed {}, probs = {}".format(i, j, circuit([np.pi/2, np.pi])))

For 1 samples with seed 0, probs = [0. 0. 1. 0.]
For 1 samples with seed 2, probs = [0. 1. 0. 0.]
For 10 samples with seed 0, probs = [0.  0.3 0.7 0. ]
For 10 samples with seed 2, probs = [0.  0.8 0.2 0. ]
For 100 samples with seed 0, probs = [0.   0.51 0.49 0.  ]
For 100 samples with seed 2, probs = [0.   0.56 0.44 0.  ]
For 1000 samples with seed 0, probs = [0.    0.517 0.483 0.   ]
For 1000 samples with seed 2, probs = [0.    0.534 0.466 0.   ]


Next we define our cost function for learning by setting the desired probability array [0, 0.5, 0.5, 0] as our target and use squre loss.

In [73]:
def cost(x):

  return sum((np.array([0, 0.5, 0.5, 0]) - np.array(circuit(x)))**2)

Before attempting to find the parameters with only one sample per iteration we can use some intution to consider why this will not work well. After each measurement, the cost is bounded below by 0.5. We use a very small number of steps and check the cost at each step to test our intuition. Our optimizer is gradient descent.

In [11]:
# initialise the optimizer with stepsize
opt = qml.GradientDescentOptimizer(1)

# set the number of samples
dev.shots = 1

# set the number of steps
steps = 10

# set seed for reproducibility 
np.random.seed(0)

# initialize the parameter to [0,0]
init_params = np.array([0,0])
params = init_params

for i in range(steps):
# update the circuit parameters after each step relative to cost & params
  params = opt.step(cost, params)
  print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
  print("Current rotation angles: {}".format(params))
print("Optimized rotation angles: {}".format(params))

Cost after step     1:  0.5000000
Current rotation angles: [0.  1.5]
Cost after step     2:  0.5000000
Current rotation angles: [0. 3.]
Cost after step     3:  0.5000000
Current rotation angles: [1. 3.]
Cost after step     4:  0.5000000
Current rotation angles: [1. 2.]
Cost after step     5:  0.5000000
Current rotation angles: [2.5 3.5]
Cost after step     6:  0.5000000
Current rotation angles: [1.5 3.5]
Cost after step     7:  0.5000000
Current rotation angles: [0.5 2.5]
Cost after step     8:  1.5000000
Current rotation angles: [0.5 2. ]
Cost after step     9:  1.5000000
Current rotation angles: [1.5 1.5]
Cost after step    10:  0.5000000
Current rotation angles: [1.5 3. ]
Optimized rotation angles: [1.5 3. ]


The cost can either be 0.5 or 1.5 depending on whether the measurement is one of [01, 10] or not respectively.

Amusingly after 1

In [83]:
# initialise the optimizer with stepsize
opt = qml.GradientDescentOptimizer(1)

# set the number of samples
dev.shots = 1

# set the number of steps
steps = 200

# set seed for reproducibility 
np.random.seed(0)

# initialize the parameter to [0,0]
init_params = np.array([0,0])
params = init_params

# explicitly state stepsize to make use of update_stepsize method
stepsize = 1

for i in range(steps):
# update the circuit parameters after each step relative to cost & params
  params = opt.step(cost, params)
  print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
  print("Current rotation angles: {}".format(params))
  if (i + 1) % 10 == 0:
      opt.update_stepsize(stepsize/2)
print("Optimized rotation angles: {}".format(params))

Cost after step     1:  0.5000000
Current rotation angles: [0.  1.5]
Cost after step     2:  0.5000000
Current rotation angles: [0. 3.]
Cost after step     3:  0.5000000
Current rotation angles: [1. 3.]
Cost after step     4:  0.5000000
Current rotation angles: [1. 2.]
Cost after step     5:  0.5000000
Current rotation angles: [2.5 3.5]
Cost after step     6:  0.5000000
Current rotation angles: [1.5 3.5]
Cost after step     7:  0.5000000
Current rotation angles: [0.5 2.5]
Cost after step     8:  1.5000000
Current rotation angles: [0.5 2. ]
Cost after step     9:  1.5000000
Current rotation angles: [1.5 1.5]
Cost after step    10:  0.5000000
Current rotation angles: [1.5 3. ]
Cost after step    11:  0.5000000
Current rotation angles: [2. 3.]
Cost after step    12:  0.5000000
Current rotation angles: [1.5 3. ]
Cost after step    13:  0.5000000
Current rotation angles: [1. 3.]
Cost after step    14:  0.5000000
Current rotation angles: [1.5  2.75]
Cost after step    15:  0.5000000
Current 

Next lets try 10 samples

In [13]:
# initialise the optimizer with stepsize
opt = qml.GradientDescentOptimizer(stepsize=1)


dev.shots = 10
# set the number of steps
steps = 1000

np.random.seed(0)
init_params = np.array([0,0])
params = init_params
stepsize = 1

for i in range(steps):
# update the circuit parameters after each step relative to cost & params
  params = opt.step(cost, params)
  if (i + 1) % 100 == 0:
    print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
    print("Current rotation angles: {}".format(params))
    if (i + 1) % 200 == 0:
      opt.update_stepsize(stepsize/10)
print("Optimized rotation angles: {}".format(params))

Cost after step   100:  0.3200000
Current rotation angles: [1.71 3.17]
Cost after step   200:  0.0800000
Current rotation angles: [1.79 3.41]
Cost after step   300:  0.0200000
Current rotation angles: [1.546 3.293]
Cost after step   400:  0.0800000
Current rotation angles: [1.557 3.253]
Cost after step   500:  0.0800000
Current rotation angles: [1.541 3.19 ]
Cost after step   600:  0.0800000
Current rotation angles: [1.541 3.275]
Cost after step   700:  0.0000000
Current rotation angles: [1.627 3.294]
Cost after step   800:  0.0200000
Current rotation angles: [1.593 3.274]
Cost after step   900:  0.0200000
Current rotation angles: [1.531 3.205]
Cost after step  1000:  0.0200000
Current rotation angles: [1.451 3.244]
Optimized rotation angles: [1.451 3.244]


L

In [ ]:
dev.shots = 10**6
np.random.seed(0)
circuit(
    



We instantiate a second device for theoretical results with the given parameters (by not setting the analytic argument to False).

In [90]:
dev2 = qml.device("default.qubit", wires=2)

@qml.qnode(dev2)
def circuit2(params):
  qml.RY(params[0], wires=0)
  qml.RY(params[1], wires=1)
  qml.CNOT(wires=[0,1])
  return qml.probs(wires=[0,1])

In [ ]:
circuit2()

In [ ]:
# initialise the optimizer with stepsize
opt = qml.GradientDescentOptimizer(stepsize=1)


dev.shots = 1000
# set the number of steps
steps = 100000

np.random.seed(0)
init_params = np.array([0,0])
params = init_params
stepsize = 1

for i in range(steps):
# update the circuit parameters after each step relative to cost & params
  params = opt.step(cost, params)
  if (i + 1) % 2000 == 0:
    print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
    print("Current rotation angles: {}".format(params))
    if (i + 1) % 10000 == 0:
      opt.update_stepsize(stepsize/5)
print("Optimized rotation angles: {}".format(params))

Cost after step  2000:  0.0032000
Current rotation angles: [ 1.604317 -3.11702 ]
Cost after step  4000:  0.0008820
Current rotation angles: [ 1.60304  -3.124372]
Cost after step  6000:  0.0015680
Current rotation angles: [ 1.592761 -3.077045]
Cost after step  8000:  0.0001460
Current rotation angles: [ 1.550887 -3.064223]
Cost after step 10000:  0.0015680
Current rotation angles: [ 1.605596 -3.098199]
Cost after step 12000:  0.0005180
Current rotation angles: [ 1.5562    -3.1109554]
Cost after step 14000:  0.0003140
Current rotation angles: [ 1.584589  -3.1178584]
Cost after step 16000:  0.0002420
Current rotation angles: [ 1.5800056 -3.1130704]
Cost after step 18000:  0.0000320
Current rotation angles: [ 1.5619176 -3.1127482]
Cost after step 20000:  0.0000320
Current rotation angles: [ 1.5690864 -3.1085846]
Cost after step 22000:  0.0000080
Current rotation angles: [ 1.5521638 -3.1056304]
Cost after step 24000:  0.0005780
Current rotation angles: [ 1.5603794 -3.1095122]
Cost after ste

In [ ]:
# initialise the optimizer with stepsize
opt = qml.GradientDescentOptimizer(stepsize=1)


dev.shots = 1000
# set the number of steps
steps = 100000

np.random.seed(1)
init_params = np.array([0,0])
params = init_params
stepsize = 1

for i in range(steps):
# update the circuit parameters after each step relative to cost & params
  params = opt.step(cost, params)
  if (i + 1) % 2000 == 0:
    print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
    print("Current rotation angles: {}".format(params))
    if (i + 1) % 10000 == 0:
      opt.update_stepsize(stepsize/5)
print("Optimized rotation angles: {}".format(params))

Cost after step  2000:  0.0008820
Current rotation angles: [1.540636 3.089585]
Cost after step  4000:  0.0013020
Current rotation angles: [1.50942  3.092861]
Cost after step  6000:  0.0005120
Current rotation angles: [1.598903 3.104701]
Cost after step  8000:  0.0004500
Current rotation angles: [1.552879 3.096904]
Cost after step 10000:  0.0000320
Current rotation angles: [1.605441 3.102533]
Cost after step 12000:  0.0000720
Current rotation angles: [1.5786776 3.1082132]
Cost after step 14000:  0.0002000
Current rotation angles: [1.561366  3.1161424]
Cost after step 16000:  0.0000500
Current rotation angles: [1.5846342 3.1156558]
Cost after step 18000:  0.0000080
Current rotation angles: [1.5610272 3.1204464]
Cost after step 20000:  0.0002420
Current rotation angles: [1.5448584 3.1183548]
Cost after step 22000:  0.0000720
Current rotation angles: [1.5589036 3.120155 ]
Cost after step 24000:  0.0000500
Current rotation angles: [1.5708318 3.1278104]
Cost after step 26000:  0.0024500
Curr

In [ ]:
# initialise the optimizer with stepsize
opt = qml.GradientDescentOptimizer(stepsize=1)


dev.shots = 100
# set the number of steps
steps = 100000

np.random.seed(0)
init_params = np.array([0,0])
params = init_params
stepsize = 1

for i in range(steps):
# update the circuit parameters after each step relative to cost & params
  params = opt.step(cost, params)
  if (i + 1) % 2000 == 0:
    print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
    print("Current rotation angles: {}".format(params))
    if (i + 1) % 10000 == 0:
      opt.update_stepsize(stepsize/10)
print("Optimized rotation angles: {}".format(params))

Cost after step  2000:  0.0128000
Current rotation angles: [1.5158 3.3335]
Cost after step  4000:  0.0008000
Current rotation angles: [1.576  3.1583]
Cost after step  6000:  0.0002000
Current rotation angles: [1.5508 3.2314]
Cost after step  8000:  0.0200000
Current rotation angles: [1.4751 3.2552]
Cost after step 10000:  0.0050000
Current rotation angles: [1.65   3.1294]
Cost after step 12000:  0.0002000
Current rotation angles: [1.56315 3.13952]
Cost after step 14000:  0.0032000
Current rotation angles: [1.56619 3.14623]
Cost after step 16000:  0.0008000
Current rotation angles: [1.53392 3.16601]
Cost after step 18000:  0.0050000
Current rotation angles: [1.56387 3.12983]
Cost after step 20000:  0.0128000
Current rotation angles: [1.57119 3.12066]
Cost after step 22000:  0.0032000
Current rotation angles: [1.5731  3.14696]
Cost after step 24000:  0.0032000
Current rotation angles: [1.52401 3.14683]
Cost after step 26000:  0.0128000
Current rotation angles: [1.55057 3.13083]
Cost afte

In [57]:
dev.shots = 10**6
np.random.seed(0)
circuit([1.58733, 3.10942])

array([1.17000e-04, 4.90871e-01, 5.08864e-01, 1.48000e-04])

In [58]:
dev.shots = 10**6
np.random.seed(0)
circuit([1.57107, 3.16395])

array([6.10000e-05, 4.99002e-01, 5.00869e-01, 6.80000e-05])

In [59]:
dev.shots = 10**6
np.random.seed(0)
circuit([1.57322, 3.09628])

array([2.47000e-04, 4.97769e-01, 5.01706e-01, 2.78000e-04])

In [37]:
dev.shots = 1000000
circuit([1.451, 3.244])

array([0.001485, 0.558185, 0.439169, 0.001161])

In [51]:
circuit2([1.451, 3.244])

array([0.00146629, 0.55828871, 0.43909177, 0.00115323])

In [60]:
circuit2([1.58733, 3.10942])

array([1.27234868e-04, 4.91606305e-01, 5.08134947e-01, 1.31512722e-04])

In [61]:
circuit2([1.57107, 3.16395])

array([6.24616660e-05, 4.99800702e-01, 5.00074341e-01, 6.24958635e-05])

In [63]:
circuit2([1.57322, 3.09628])

array([2.55988721e-04, 4.98532176e-01, 5.00954603e-01, 2.57232600e-04])

In [65]:
circuit2([1.53798, 3.12897])

array([2.05696164e-05, 5.16384649e-01, 4.83575519e-01, 1.92627006e-05])

In [67]:
circuit2([1.55117, 3.16505])

array([7.01275014e-05, 5.09742406e-01, 4.90120039e-01, 6.74279661e-05])